<center><h1>Trigger Word Detection</h1></center>

Importing Libraries

In [1]:
import model
from keras.models import load_model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Defining Constants

In [2]:
Tx = 5511 # Use 1101 for 2 sec
n_freq = 101

In [3]:
Ty = 1375 # Use 272 for 2 sec

Loading Pre-Trained Model

In [4]:
model = load_model('pre-trained models/activate_model.h5')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/usr/local/lib/python3.5/dist-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Checking for Trigger Word

In [5]:
def detect_trigger_word_spectrum(x):
    x = x.swapaxes(0, 1)
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)
    return predictions.reshape(-1)

In [6]:
def has_new_trigger_word(predictions, chunk_duration, feed_duration, threshold=0.5):
    predictions = predictions > threshold
    chunk_predictions_samples = int(len(predictions) * chunk_duration / feed_duration)
    chunk_predictions = predictions[-chunk_predictions_samples:]
    level = chunk_predictions[0]
    for pred in chunk_predictions:
        if pred > level:
            return True
        else:
            level = pred
    return False

<center><h2>Recording from MIC</h2></center>

Importing Libraries

In [7]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plot

Defining Constants

In [8]:
chunk_duration = None
fs = None
chunk_samples = None

feed_duration = None
feed_samples = None

Intializing above Constants

In [9]:
def initialize_mic():
    global chunk_duration, fs, chunk_samples, feed_duration, feed_samples
    chunk_duration = 0.5
    fs = 44100
    chunk_samples = int(fs * chunk_duration)
    
    feed_duration = 10
    feed_samples = int(fs * feed_duration)
    
    assert feed_duration/chunk_duration == int(feed_duration/chunk_duration)

Calling the Initialing Function

In [10]:
initialize_mic()

Finding the Spectrum of our Data

In [11]:
def get_spectrogram(data):
    nfft = 200
    fs = 8000
    noverlap = 120
    nchannels = data.ndim
    if nchannels == 1:
        pxx, _, _ = mlab.specgram(data, nfft, fs, noverlap=noverlap)
    elif nchannels == 2:
        pxx, _, _ = mlab.specgram(data[:, 0], nfft, fs, noverlap=noverlap)
    return pxx

Plotting the Spectrum of our Data

In [12]:
def plot_spectrogram(data):
    nfft = 200
    fs = 8000
    noverlap = 12
    nchannels = data.ndim
    if nchannels == 1:
        pxx, _, _ = plot.specgram(data, nfft, fs, noverlap=noverlap)
    elif nchannels == 2:
        pxx, _, _ = plot.specgram(data[:, 0], nfft, fs, noverlap=noverlap)
    return pxx

<center><h2>Audio Stream</h2></center>

Importing Libraries

In [13]:
import pyaudio
from queue import Queue
import sys
import numpy as np
import time

Getting the Audio Input Stream

In [14]:
def get_audio_input_stream(callback):
    global fs, chunk_samples
    stream = pyaudio.PyAudio().open(
        format=pyaudio.paInt16,
        channels=1,
        rate=fs,
        input=True,
        frames_per_buffer=chunk_samples,
        input_device_index=5,
        stream_callback=callback)
    return stream

Defining Constants

In [15]:
run = True
q = None
data = None
timeout = None
silence_threshold = None

Intializing above Constants

In [16]:
def initialize_audio_stream():
    global run, q, data, timeout, silence_threshold
    q = Queue()
    run = True
    silence_threshold = 100
    timeout = time.time() + 0.5*60
    data = np.zeros(feed_samples, dtype='int16')
    return None

Calling the Initialing Function

In [17]:
initialize_audio_stream()

Function for Audio Input Callback

In [18]:
def callback(in_data, frame_count, time_info, status):
    global run, timeout, data, silence_threshold    
    if time.time() > timeout:
        run = False        
    data0 = np.frombuffer(in_data, dtype='int16')
    if np.abs(data0).mean() < silence_threshold:
        sys.stdout.write('-')
        return (in_data, pyaudio.paContinue)
    else:
        sys.stdout.write('.')
    data = np.append(data,data0)    
    if len(data) > feed_samples:
        data = data[-feed_samples:]
        q.put(data)
    return (in_data, pyaudio.paContinue)

Starting the Detection

In [24]:
def start():
    stream = get_audio_input_stream(callback)
    stream.start_stream()
    print(chunk_duration, feed_duration)
    global run
    try:
        while run:
            data = q.get()
            spectrum = get_spectrogram(data)
            preds = detect_trigger_word_spectrum(spectrum)
            new_trigger = has_new_trigger_word(preds, chunk_duration, feed_duration)
            if new_trigger:
                sys.stdout.write('1')
    except (KeyboardInterrupt, SystemExit):
        stream.stop_stream()
        stream.close()
        timeout = time.time()
        run = False

In [26]:
start()

..0.5 10
.......

In [27]:
def stop():
    global run
    run = False
    stream.stop_stream()

.....

In [28]:
stop()

..........................................................................--.............--...--........................................................................................................................